# Support Vector Machine

In [1]:
# Zeige Plot in Notebook
%matplotlib inline
# Importiere Bibliotheken
import sys
import io
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions

In [2]:
# Kodierte Daten einlesen
data_labeled = pd.read_pickle('data_modified/tweets_labeled.pkl')
data_labeled.head()

,User,Tweet,Retweet,RT_from,Orig_Tweet,Tweet_Participants,Reply,Text,Tags,Lang,Link,Label
Datetime,,,,,,,,,,,,
2018-01-18 17:07:00,@GeroldSchlegel,RT @redder66: @NatalieRickli @NoBillag Die Unt...,True,@redder66,False,"[@NatalieRickli, @NoBillag, @SVPch]",False,Die Unterstützung der für die #NoBillag ergibt...,"[NoBillag, NeinzuNoBillag]",de,[],1
2017-12-23 13:53:00,@derlamentierer,@ProBillag auch sie haben das konzept von soli...,False,NaN,False,[@ProBillag],True,auch sie haben das konzept von solidarität nic...,"[nobillag, jazunobillag, NeinzuNoBillag]",de,[https://t.co/iFLeHBuSFS],6
2017-12-05 19:47:00,@FlorianSchwab,RT @VitalSteiner: Was die Gegner von #NoBillag...,True,@VitalSteiner,False,[],False,Was die Gegner von #NoBillag mit solchen Bilde...,[NoBillag],de,[https://t.co/IKOSeftUlm],6
2018-01-17 07:08:00,@Schweizer2018,RT @realDanielVozar: Wenn #NoBillag abgelehnt ...,True,@realDanielVozar,False,[@LHeimgartner],False,"Wenn #NoBillag abgelehnt wird, wird sich gar n...","[NoBillag, JazuNoBillag]",de,[],5
2018-02-19 09:52:00,@PilgramsWords,RT @_macmike: #StarkesStück https://t.co/2L7uG...,True,@_macmike,False,"[@Blickch, @20min, @bazonline]",False,#StarkesStück #AfD #SVP #ARD #ZDF #ORF #SRF #...,"[StarkesStück, AfD, SVP, ARD, ZDF, ORF, SRF, s...",de,"[https://t.co/2L7uGmmVH0, https://t.co/5Wf6qJ1...",0


In [3]:
# Funktionen definieren um Stoppworte zu entfernen & Daten weiter zu bereinigen
nltk.download('stopwords')
stopword_set = set(stopwords.words("german"))
def preprocess(raw_text):
    res = raw_text
    res = res.replace('ä', 'ae')
    res = res.replace('ö', 'oe')
    res = res.replace('ü', 'ue')
    res = res.replace('Ä', 'Ae')
    res = res.replace('Ö', 'Oe')
    res = res.replace('Ü', 'Ue')
    res = res.replace('ß', 'ss')
    stopword_set = set(stopwords.words("german"))
    return " ".join([i for i in re.sub(r'[^a-zA-Z\s]', "", res).lower().split() if i not in stopword_set])

[nltk_data] Downloading package stopwords to C:\Users\Sven
[nltk_data]     Cornehls\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Daten bereinigen und X und y Variablen legen
texts=[]
for txt in data_labeled['Text']:
    txt=preprocess(txt)
    texts.append(txt)
labels = data_labeled['Label'].values

In [5]:
# TF-IDF document-term matrix erstellen
from sklearn.feature_extraction.text import CountVectorizer

count_vect_x = CountVectorizer(min_df=1)
X_counts= count_vect_x.fit_transform(texts)
X_counts.shape

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)
X_tfidf

<1034x5301 sparse matrix of type '<class 'numpy.float64'>'
	with 12316 stored elements in Compressed Sparse Row format>

In [6]:
# test and train Daten definieren
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.5, random_state=14)

In [7]:
# SVC Modell mit anderem Kernel ausführen
from sklearn import svm
model1 = svm.SVC(kernel = 'linear')
model1.fit(X_train, y_train)
# Genauigkeit der neuen Vorhersage ermitteln
print("Accuracy von SVM mit linearen kernel: {}".format(model1.score(X_test, y_test)))

Accuracy von SVM mit linearen kernel: 0.42359767891682787


In [8]:
# SVC Modell mit anderem Kernel ausführen
from sklearn import svm
model2 = svm.SVC(kernel = 'poly')
model2.fit(X_train, y_train)
# Genauigkeit der neuen Vorhersage ermitteln
print("Accuracy von SVM mit polynomial kernel: {}".format(model2.score(X_test, y_test)))

Accuracy von SVM mit polynomial kernel: 0.35589941972920697


In [9]:
# SVC Modell mit anderem Kernel ausführen
from sklearn import svm
model3 = svm.SVC(kernel = 'rbf')
model3.fit(X_train, y_train)
# Genauigkeit der neuen Vorhersage ermitteln
print("Accuracy von SVM mit RBF kernel: {}".format(model3.score(X_test, y_test)))

Accuracy von SVM mit RBF kernel: 0.35589941972920697


## Nächster Schritt nach Entscheidung für SVM:

### Parameter 'tunen'

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# Parameter für GridSearchCV definieren
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']


for score in scores:
    print("# Parameter tunen für %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Beste Parameter für Datensatz:")
    print()
    print(clf.best_params_)
    print()
    print("Scorest:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailierter classification report:")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 10, 'kernel': 'linear'}

Grid scores on development set:

0.034 (+/-0.001) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.034 (+/-0.001) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.034 (+/-0.001) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.034 (+/-0.001) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.101 (+/-0.110) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.034 (+/-0.001) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.434 (+/-0.165) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.101 (+/-0.110) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.432 (+/-0.220) for {'C': 1, 'kernel': 'linear'}
0.459 (+/-0.186) for {'C': 10, 'kernel': 'linear'}
0.449 (+/-0.188) for {'C': 100, 'kernel': 'linear'}
0.449 (+/-0.188) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full ev

In [11]:
# lineares SVC Modell mit neuen Parametern ausführen
from sklearn import svm
model4 = svm.SVC(kernel = 'linear', C=10)
model4.fit(X_train, y_train)
# Genauigkeit der neuen Vorhersage ermitteln
print("Accuracy von SVM mit linearen kernel: {}".format(model4.score(X_test, y_test)))

Accuracy von SVM mit linearen kernel: 0.4410058027079304


In [12]:
# lineares SVC Modell mit neuen Parametern ausführen
from sklearn import svm
model5 = svm.SVC(kernel = 'rbf', C=1000, gamma=0.001)
model5.fit(X_train, y_train)
# Genauigkeit der neuen Vorhersage ermitteln
print("Accuracy von SVM mit linearen kernel: {}".format(model5.score(X_test, y_test)))

Accuracy von SVM mit linearen kernel: 0.4584139264990329


In [30]:
# Erneut Classification report anschauen
from sklearn.metrics import classification_report
print(classification_report(y_test, model5.predict(X_test)))

             precision    recall  f1-score   support

          0       0.44      0.92      0.60       184
          1       0.65      0.37      0.47        54
          2       0.47      0.25      0.33        36
          3       0.43      0.20      0.27        45
          4       1.00      0.11      0.20        18
          5       0.55      0.14      0.22        43
          6       0.60      0.12      0.20        50
          7       0.00      0.00      0.00        20
          8       0.42      0.24      0.30        67

avg / total       0.49      0.46      0.39       517



## Vorhersage der Tweets durchführen

In [13]:
# Nicht Kodierte Daten einlesen
data_unlabeled = pd.read_pickle('data_modified/tweets_unlabeled.pkl')
data_unlabeled.head()

,User,Tweet,Retweet,RT_from,Orig_Tweet,Tweet_Participants,Reply,Text,Tags,Lang,Link
Datetime,,,,,,,,,,,
2017-10-18 20:32:00,@RolfMll35367465,#NoBillag\nDie SRG war immer schon ein linker ...,False,NaN,True,[],False,#NoBillag Die SRG war immer schon ein linker h...,[NoBillag],de,[https://t.co/deq4ccfpND]
2018-02-16 12:29:00,@DeSpell_,Beitrag zur #nobillag-Debatte: Service-public-...,False,NaN,True,[],False,Beitrag zur #nobillag-Debatte: Service-public-...,"[nobillag, ORF]",de,[https://t.co/VtscRYniBp]
2017-12-20 10:32:00,@RiesenChristian,"@ginsig Passt nicht zusammen, dass Sie einerse...",False,NaN,False,[@ginsig],True,"Passt nicht zusammen, dass Sie einerseits auf ...","[Zwangsgebühr, Billag, NoBillag]",de,[]
2018-03-01 09:14:00,@Luv_Lee1951,‼️🇨🇭...und wird z.B. von einer #BILLAG in der ...,False,NaN,True,[],False,‼️🇨🇭...und wird z.B. von einer #BILLAG in der ...,"[BILLAG, Schweiz, nobillag, SVP]",de,[https://t.co/8xZkQQr4iE]
2018-02-01 12:05:00,@_macmike,@Chat_Atkins @Blickch @20min @bazonline https:...,False,NaN,False,"[@Chat_Atkins, @Blickch, @20min, @bazonline, @...",True,#MeToo #AfD #SVP #ARD #ZDF #ORF #SRF #srfaren...,"[MeToo, AfD, SVP, ARD, ZDF, ORF, SRF, srfarena...",de,[https://t.co/sq0F1fxMMi]


In [14]:
# Daten bereinigen und in X legen
texts_new=[]
for txt in data_unlabeled['Text']:
    txt=preprocess(txt)
    texts_new.append(txt)

In [18]:
# TF-IDF document-term matrix erstellen
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


count_vect_x_unlabeled = CountVectorizer(min_df=1)
X_counts_unlabeled= count_vect_x.transform(texts_new)
X_counts_unlabeled.shape


tfidf_vectorizer = TfidfVectorizer()
X_tfidf_labeled=tfidf_vectorizer.fit_transform(texts)
X_tfidf_unlabeled = tfidf_vectorizer.transform(texts_new)
X_tfidf_unlabeled

<102390x5301 sparse matrix of type '<class 'numpy.float64'>'
	with 832242 stored elements in Compressed Sparse Row format>

In [20]:
# Voraussagen mit unkodiertem Datensatz machen
predicted_frames = model5.predict(X_tfidf_unlabeled)

In [44]:
# Voraussagen in Datensatz legen
data_unlabeled['Label'] = predicted_frames

,User,Tweet,Retweet,RT_from,Orig_Tweet,Tweet_Participants,Reply,Text,Tags,Lang,Link,Label,Label2
Datetime,,,,,,,,,,,,,
2017-10-18 20:32:00,@RolfMll35367465,#NoBillag\nDie SRG war immer schon ein linker ...,False,NaN,True,[],False,#NoBillag Die SRG war immer schon ein linker h...,[NoBillag],de,[https://t.co/deq4ccfpND],5,5
2018-02-16 12:29:00,@DeSpell_,Beitrag zur #nobillag-Debatte: Service-public-...,False,NaN,True,[],False,Beitrag zur #nobillag-Debatte: Service-public-...,"[nobillag, ORF]",de,[https://t.co/VtscRYniBp],0,0
2017-12-20 10:32:00,@RiesenChristian,"@ginsig Passt nicht zusammen, dass Sie einerse...",False,NaN,False,[@ginsig],True,"Passt nicht zusammen, dass Sie einerseits auf ...","[Zwangsgebühr, Billag, NoBillag]",de,[],6,6
2018-03-01 09:14:00,@Luv_Lee1951,‼️🇨🇭...und wird z.B. von einer #BILLAG in der ...,False,NaN,True,[],False,‼️🇨🇭...und wird z.B. von einer #BILLAG in der ...,"[BILLAG, Schweiz, nobillag, SVP]",de,[https://t.co/8xZkQQr4iE],7,7
2018-02-01 12:05:00,@_macmike,@Chat_Atkins @Blickch @20min @bazonline https:...,False,NaN,False,"[@Chat_Atkins, @Blickch, @20min, @bazonline, @...",True,#MeToo #AfD #SVP #ARD #ZDF #ORF #SRF #srfaren...,"[MeToo, AfD, SVP, ARD, ZDF, ORF, SRF, srfarena...",de,[https://t.co/sq0F1fxMMi],0,0
2017-12-01 20:00:00,@GebuehrNudel,RT @NoBillag: Jetzt das #NoBillag Crowdfunding...,True,@NoBillag,False,[],False,Jetzt das #NoBillag Crowdfunding mit einer SMS...,"[NoBillag, DavidgegeGoliath, NoBillagWeekend]",de,[https://t.co/6mR8bJvcqC],0,0
2017-12-23 16:28:00,@hansjaaggi,NZZ: Noch ein Kommentar zur #SRG -Zerschlagung...,False,NaN,True,[],False,NZZ: Noch ein Kommentar zur #SRG -Zerschlagung...,"[SRG, NEINzuNoBillag]",de,[https://t.co/lHq3gKSVz2],5,5
2017-12-17 00:02:00,@louisdana,@_Matthieu_ @NordmannRoger Je lis tes tweets s...,False,NaN,False,"[@_Matthieu_, @NordmannRoger]",True,Je lis tes tweets sur #NoBillag. Tu prendras p...,[NoBillag],fr,[],0,0
2018-03-17 16:06:00,@WernerBechtel,Nach der #NoBillag #abst18 \nist für mich auch...,False,NaN,True,[],False,Nach der #NoBillag #abst18 ist für mich auch #...,"[NoBillag, abst18, srfarena, Hallosrf]",de,[https://t.co/RNX901BpL2],0,0


In [ ]:
# Neuen Datensatz abspeichern
data_labeled.to_pickle('data_modified/tweets_predicted.pkl')

### Testcode (wird nicht mehr ausgeführt)